In [1]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms python-terrier

In [2]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="notebook")

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pathlib import Path
from examples.trec_util import TrecTrack

track = TrecTrack(
    28, "deep", "msmarco-passage/trec-dl-2019",
    base_results_dir=Path("/home/heinrich/Development/webis/trec-system-runs"),
    # base_results_dir=Path("/path/to/trec-system-runs"),  # Path with TREC runs in folders like ./trec-28/deep/
)
retrieval_systems = [result % 10 for result in track.results]

Read results: 100%|██████████| 37/37 [00:15<00:00,  2.44run/s]


In [4]:
from ir_axioms.axiom import TFC1, TFC3, LNC1, OriginalAxiom

# Define axiom for reranker and permutations.
axiom = TFC1() + TFC3() + LNC1() + (OriginalAxiom() * 2.0)

In [5]:
from ir_axioms.backend.pyterrier.transformers import AxiomaticReranker

reranked_retrieval_systems = [
    retrieval_system >> AxiomaticReranker(
        axiom=axiom,
        index=track.index,
        dataset=track.dataset_name,
        verbose=True
    )
    for retrieval_system in retrieval_systems
]

In [6]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG

# noinspection PyTypeChecker
experiment = Experiment(
    [*retrieval_systems, *reranked_retrieval_systems],
    track.dataset.get_topics(),
    track.dataset.get_qrels(),
    [nDCG @ 10],
)
experiment

AxiomaticReranker:   4%|▍         | 8/200 [00:10<05:00,  1.57s/query]

21:33:39.003 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading lookup file directly from disk
21:33:39.055 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk


AxiomaticReranker:   4%|▍         | 9/200 [00:12<04:36,  1.45s/query]

21:33:39.156 [main] ERROR org.terrier.structures.Index - Couldn't load an index structure called document
java.lang.OutOfMemoryError: Java heap space
21:33:39.249 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading lookup file directly from disk
21:33:39.285 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
21:33:39.321 [main] ERROR org.terrier.structures.Index - Couldn't load an index structure called document
java.lang.OutOfMemoryError: Java heap space
21:33:39.383 [main] ERROR org.terrier.structures.Index - Couldn't load an index structure called document
java.lang.OutOfMemoryError: Java heap space


AxiomaticReranker:   4%|▍         | 9/200 [00:12<04:24,  1.38s/query]


JavaException: JVM exception occurred: Could not load an index for ref /home/heinrich/Development/webis/ir_axioms/examples/data/indices/msmarco-passage/trec-dl-2019, even though IndexLoader org.terrier.structures.IndexOnDisk$DiskIndexLoader could support that type of index. It may be your ref had a wrong location; Terrier logs may have more information. java.lang.IllegalArgumentException